In [7]:
#Importing Python Libraries that will be used

from bs4 import BeautifulSoup
import csv
import requests

#Defining search term that will be used to get data using hiperlink and adding page reference. Data will be divided by room numbers of the appartments

def get_url(search_term):
    
    template='https://bina.az/baki/alqi-satqi/menziller'
    url=template+'/'+search_term+'-otaqli'
    
    url+='?page={}'
    
    return url

#Defining each item in the page from the html code of the webpage, in this case 'ID' of the each Ad, 'Price' of the appartment, 'Location', 'Room' Number, 'Area' of the appartment, 'Floor', 'Store' of the Building and 'Link' of the Ads. 

def extract_record(item):
    
    ID=format(item.find('a', "item_link")).split("/", 1)[1].split("/", 1)[1].split('"', 1)[0]
    Price=item.find('span',"price-val").text
    Location=item.find('div',"location").text
    Room=item.find('ul', "name").text.split(" otaqlı", 1)[0]
    Area=item.find('ul', "name").text.split(" otaqlı", 1)[1].split(" m", 1)[0]
    Floor=item.find('ul', "name").text.split(" otaqlı", 1)[1].split(" m²", 1)[1].split("/", 1)[0]
    
    if Floor!='':
        Store=item.find('ul', "name").text.split(" otaqlı", 1)[1].split(" m²", 1)[1].split("/", 1)[1].split(" mərtəbə", 1)[0]
    else:
        Store=''
        
    Date_Time=item.find('div', "city_when").text.split(", ", 1)[1]
    Link='https://bina.az/items/'+ID
    
    result=(ID, Price, Location, Room, Area, Floor,Store, Date_Time, Link)
    
    return result

#Putting data into the table format

def main(search_term):
    
    records=[]
    url=get_url(search_term)
    
#Getting data from the first 100 pages of the website
    
    for page in range(1,100):
        
        headers = {"User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/97.0.4692.71 Safari/537.36", "Accept-Encoding":"gzip, deflate", "Accept":"text/html,application/xhtml+xml,application/xml;q=0.9,*/*;q=0.8", "DNT":"1","Connection":"close", "Upgrade-Insecure-Requests":"1"}
        
        page = requests.get(url.format(page), headers=headers)
        
        soup1 = BeautifulSoup(page.content, "html.parser")

        results=soup1.find_all('div',"items-i")
        
        for item in results:
            record=extract_record(item)
            if record:
                records.append(record)
                
#Saving the data into the csv file according to room number (result1)

        with open('results1.csv', 'w', newline='', encoding='utf-8-sig') as f:
            writer=csv.writer(f)
            writer.writerow(['ID','Price', 'Location', 'Room', 'Area', 'Floor', 'Store', "Date_Time", "Link"])
            writer.writerows(records)
        

In [8]:
#Running the function to extract the data and to put into the csv file. Each time function is run from '1' to '4' to get 4 csv tables according to Room number which are will be combined in Power BI.

main('1')